# Sonata and NWB

The sonata [simulation results format](https://github.com/AllenInstitute/sonata/blob/master/docs/SONATA_DEVELOPER_GUIDE.md#output-file-formats) are compatable with NWB, meaning you can use pynwb and matnwb APIs to read sonata files generated during the run of your simulation. Here we will show how to use pynwb to read sonata files.

Doing so you'll need [pynwb](https://github.com/NeurodataWithoutBorders/pynwb) and the [ndx-simulation-output](https://github.com/bendichter/ndx-simulation-output) nwb extension, a format developed to be compatable with sonata multi-compartment files. The easiest way to install both is with the pip command:
```bash
pip install ndx-simulation-output
```


## Converting sonata to pynwb

To start with we generate the required simulation output hdf5 files using one of the many tools that support sonata. The [examples page](https://github.com/AllenInstitute/sonata/tree/master/examples) contains a number of networks and scripts/tutorials on how to run a simulation using your prefered tools. We've also included some pre-generated examples in the *example_data/* folder. In particularly there are three example files:
 * membrane_potential.h5 - An example multicompartment report
 * spikes.h5 - An example spikes report
 * ecp.h5 - An example of extracelluar multi-channel recording

We will save these in the nwb file 'simulation_results.nwb'

In [1]:
import os
from ndx_simulation_output.io import sonata2nwb
sonata2nwb(data_path=['example_data/membrane_potential.h5',
                      'example_data/spikes.h5',
                      'example_data/ecp.h5'], 
           save_path='simulation_results.nwb',
           electrodes_file='example_data/electrodes.csv'
          )

print(os.path.exists('simulation_results.nwb'))


reading units: 100%|██████████| 5/5 [00:00<00:00, 1718.84it/s]


True


## Reading Compartment Reports

Next use pynwb to read in our converted nwb file

In [2]:
from pynwb import NWBHDF5IO
io = NWBHDF5IO('simulation_results.nwb', mode='r')
nwb = io.read()

The compartment reports are stored under acquisitions group. Here we can see the nwb file contains the **membrane_potential** data (there is also an ElectrialSeries group that we will use below for reading ecp results). 

In [3]:
print(nwb.acquisition)

vm_report = nwb.acquisition['membrane_potential']
print('node_ids: {}'.format(vm_report.compartments.id.data[()]))

{'ElectricalSeries': 
ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (1000, 15), type "<f8">
  description: no description
  electrodes: electrodes <class 'pynwb.core.DynamicTableRegion'>
  num_samples: 1000
  rate: 10000.0
  resolution: 0.0
  starting_time: 0.0
  starting_time_unit: Seconds
  unit: volt
, 'membrane_potential': 
membrane_potential <class 'abc.CompartmentSeries'>
Fields:
  comments: no comments
  compartments: compartments <class 'ndx_simulation_output.simulation_output.Compartments'>
  conversion: 1.0
  data: <HDF5 dataset "data": shape (1000, 361), type "<f8">
  description: no description
  num_samples: 1000
  rate: 10000.0
  resolution: 0.0
  starting_time: 0.0
  starting_time_unit: Seconds
  unit: mV
}
node_ids: [0 1 2 3 4]


For a given node we can find the associated compartment labels, their positions, and the time traces during the simulation:

In [4]:
node_id, element_ids, positions = nwb.acquisition['membrane_potential'].compartments[1]

unit_indices = vm_report.find_compartments(1)
vm_traces = vm_report.data[:, unit_indices][()]

print('node_id: {}'.format(node_id))
print('  compartment (element) labels: {}'.format(element_ids))
print('  positions: {}'.format(positions))
print(' data steps x compartments: {} x {}'.format(vm_traces.shape[0], vm_traces.shape[1]))
print(vm_traces)

node_id: 1
  compartment (element) labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
  positions: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
 data steps x compartments: 1000 x 50
[[1.    1.    1.    ... 1.    1.    1.   ]
 [1.001 1.001 1.001 ... 1.001 1.001 1.001]
 [1.002 1.002 1.002 ... 1.002 1.002 1.002]
 ...
 [1.997 1.997 1.997 ... 1.997 1.997 1.997]
 [1.998 1.998 1.998 ... 1.998 1.998 1.998]
 [1.999 1.999 1.999 ... 1.999 1.999 1.999]]


## Reading Spike Reports

We can also fetch the spike time simulation output for each recorded node_id

In [5]:
nwb.units.to_dataframe()


,spike_times
id,
0,"[46.44830098095446, 73.62532407151554, 99.6196..."
1,"[2.8285827476543615, 34.98136107011318, 54.701..."
2,"[38.30268119285648, 39.31566714213238, 66.6219..."
3,"[18.39203560767911, 50.4658832902191, 92.16889..."
4,"[28.938474810891595, 58.59423608546243, 63.637..."


In [9]:
# Get spikes for a single unit/node
unit_id = 0
print('spike times for unit {}'.format(unit_id))
nwb.units['spike_times'][unit_id]

spike times for unit 0


array([46.44830098, 73.62532407, 99.61962915])

## Reading Extracellular Reports

Extracellular reports in NWB are saved using the [ecephys module](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html) and can be accessed like you would with real data.


There are a number of ways to fetch information about the recording electrode channels, including as a pandas DataFrame

In [10]:
nwb.electrodes.to_dataframe()

,x,y,z,imp,location,filtering,group,group_name
id,,,,,,,,
0,0.0,0.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
1,0.0,1.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
2,0.0,2.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
3,0.0,3.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
4,0.0,4.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
5,0.0,5.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
6,0.0,6.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
7,0.0,7.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant
8,0.0,8.0,0.0,NaN,unknown,none,\nsimulated_implant <class 'pynwb.ecephys.Elec...,simulated_implant


As we saw above the data is stored in acquisitions

In [11]:
ecp_report = nwb.acquisition['ElectricalSeries']
ecp_report


ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (1000, 15), type "<f8">
  description: no description
  electrodes: electrodes <class 'pynwb.core.DynamicTableRegion'>
  num_samples: 1000
  rate: 10000.0
  resolution: 0.0
  starting_time: 0.0
  starting_time_unit: Seconds
  unit: volt

In [12]:
ecp_report.data[()]

array([[0.57293099, 0.76930203, 0.88071005, ..., 0.70047962, 0.28730622,
        0.96992367],
       [0.15170223, 0.68863223, 0.44048643, ..., 0.61865637, 0.38654594,
        0.18701311],
       [0.9690474 , 0.52246217, 0.23944365, ..., 0.61955686, 0.86474007,
        0.48352767],
       ...,
       [0.56341004, 0.31859838, 0.00289707, ..., 0.52821551, 0.02956707,
        0.34990551],
       [0.64309011, 0.8716774 , 0.20747117, ..., 0.6707939 , 0.05339673,
        0.91099735],
       [0.27379172, 0.70690807, 0.02127752, ..., 0.48369901, 0.8738186 ,
        0.14116656]])